In [1]:
# 6.2 グリッドサーチ

In [2]:
from sklearn.model_selection import ParameterGrid
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\210450279\Anaconda3\envs\rebases\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [4]:
# 再現性の確保のために乱数シードを固定（数値は何でもよい）
tf.set_random_seed(12345)

In [5]:
# 入力データ
# MNISTのワンホット表現での読み込み
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, validation_size=10000)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# 0 入力画像
x = tf.placeholder(tf.float32, name='x')

In [7]:
# 1 サイズ変更
x_1 = tf.reshape(x, [-1, 28, 28, 1])

In [8]:
# 2 畳み込み
# ランダムカーネル
k_0 = tf.Variable(tf.truncated_normal([4, 4, 1, 10], mean=0.0, stddev=0.1))
# 畳み込み
x_2 = tf.nn.conv2d(x_1, k_0, strides=[1, 3, 3, 1], padding='VALID')

In [9]:
# 3 活性化関数
x_3 = tf.nn.relu(x_2)

In [10]:
# 4 プーリング
x_4 = tf.nn.max_pool(x_3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

In [11]:
# 5 サイズ変更
x_5 = tf.reshape(x_4, [-1, 160])

In [12]:
#ドロップアウト付きの全結合
def matmul_plus_bias_with_dropout(x, w, b, p):
    return tf.matmul(tf.nn.dropout(x, keep_prob=p), w) + b

In [13]:
# 6 全結合
# 重みとバイアス
w_1 = tf.Variable(tf.zeros([160, 40]))
b_1 = tf.Variable([0.1] * 40)
#ドロップアウト率
p_1 = tf.placeholder(tf.float32, name='p_1')
# 全結合
x_6 = matmul_plus_bias_with_dropout(x_5, w_1, b_1, p_1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
# 7 活性化関数
x_7 = tf.nn.relu(x_6)

In [15]:
# 8 全結合
# 重みとバイアス
w_2 = tf.Variable(tf.zeros([40, 10]))
b_2 = tf.Variable([0.1] * 10)
#ドロップアウト率
p_2 = tf.placeholder(tf.float32, name='p_2')
# 全結合
x_8 = matmul_plus_bias_with_dropout(x_7, w_2, b_2, p_2)

In [16]:
# 9 確率化
y = tf.nn.softmax(x_8, name='y')

In [17]:
# 10 損失関数の最小化
# 正解ラベル
labels = tf.placeholder(tf.float32, name='labels')
# 損失関数（交差エントロピー）と最適化処理（Adam）
loss = -tf.reduce_sum(labels * tf.log(y + 1e-9)) / tf.cast(tf.shape(y)[0], tf.float32)
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [18]:
# 11 精度検証 (Top-1 accuracy)
prediction_match = tf.equal(tf.argmax(y, axis=1), tf.argmax(labels, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name='accuracy')

In [19]:
# パラメーター
# バッチサイズ
BATCH_SIZE = 32
# 学習回数
NUM_TRAIN = 10_000
# 学習中の出力頻度
OUTPUT_BY = 500
#ドロップアウト率
DROPOUT_PROB_1 = [0.2, 0.5, 0.8]
DROPOUT_PROB_2 = [0.2, 0.5, 0.8]

In [20]:
# -----------------------------------------------------------------------------------
# ParameterGridは、パラメータの組を返す単純なイテレータである。
# GridSearchCVは、scikit-learnが提供している機械学習モデルの最適化に使用する。
# 自作のモデルを使うこともできるが、BaseEstimatorを継承したクラスを実装する必要がある。
# -----------------------------------------------------------------------------------


# 探索空間の設定
search_space = {'DROPOUT_PROB_1': DROPOUT_PROB_1, 'DROPOUT_PROB_2': DROPOUT_PROB_2}
# 最適値を初期化
best_param = None
best_val_accuracy = 0.0
# グリッドサーチを実行
param_grid = ParameterGrid(search_space)
n_params = len(param_grid)

In [21]:
for t, param in enumerate(param_grid):
    print('[{:d}/{:d}]\tparam: {}'.format(t+1, n_params, str(param)))
    # 学習の実行
    sess.run(tf.global_variables_initializer())
    dropout_prob = {p_1: param['DROPOUT_PROB_1'], p_2: param['DROPOUT_PROB_2']}
    for i in range(NUM_TRAIN):
        batch = mnist.train.next_batch(BATCH_SIZE)
        inout = {x: batch[0], labels: batch[1]}
        optimizer.run(feed_dict={**inout, **dropout_prob})

    batch = mnist.train.next_batch(BATCH_SIZE)
    inout = {x: batch[0], labels: batch[1]}
    train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict={**inout, p_1: 1.0, p_2: 1.0})
    val_loss, val_accuracy = sess.run([loss, accuracy], feed_dict={
        x: mnist.validation.images, labels: mnist.validation.labels, p_1: 1.0, p_2: 1.0})
    print('\tstep {:d}, train_loss {:.2f}, val_loss {:.2f}, train_acc {:.2f}, val_acc {:.2f}'.format(
            NUM_TRAIN, train_loss, val_loss, train_accuracy, val_accuracy))

    # バリデーションセットに対する精度が最も高いパラメータを記録する
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_param = param

[1/9]	param: {'DROPOUT_PROB_1': 0.2, 'DROPOUT_PROB_2': 0.2}
	step 10000, train_loss 0.99, val_loss 1.03, train_acc 0.78, val_acc 0.80
[2/9]	param: {'DROPOUT_PROB_1': 0.2, 'DROPOUT_PROB_2': 0.5}
	step 10000, train_loss 0.67, val_loss 0.69, train_acc 0.84, val_acc 0.86
[3/9]	param: {'DROPOUT_PROB_1': 0.2, 'DROPOUT_PROB_2': 0.8}
	step 10000, train_loss 0.52, val_loss 0.55, train_acc 0.94, val_acc 0.88
[4/9]	param: {'DROPOUT_PROB_1': 0.5, 'DROPOUT_PROB_2': 0.2}
	step 10000, train_loss 0.74, val_loss 0.63, train_acc 0.81, val_acc 0.88
[5/9]	param: {'DROPOUT_PROB_1': 0.5, 'DROPOUT_PROB_2': 0.5}
	step 10000, train_loss 0.35, val_loss 0.28, train_acc 0.94, val_acc 0.93
[6/9]	param: {'DROPOUT_PROB_1': 0.5, 'DROPOUT_PROB_2': 0.8}
	step 10000, train_loss 0.29, val_loss 0.23, train_acc 0.88, val_acc 0.94
[7/9]	param: {'DROPOUT_PROB_1': 0.8, 'DROPOUT_PROB_2': 0.2}
	step 10000, train_loss 0.53, val_loss 0.44, train_acc 0.91, val_acc 0.92
[8/9]	param: {'DROPOUT_PROB_1': 0.8, 'DROPOUT_PROB_2': 0.5}
	s

In [22]:
print('best param: {}'.format(str(best_param)))
print('best val_acc: {:.2f}'.format(val_accuracy))

best param: {'DROPOUT_PROB_1': 0.8, 'DROPOUT_PROB_2': 0.8}
best val_acc: 0.96
